# Datum Transformation

GeodePy has the ability to **transform between datums**. Here we discuss how to change between datums **without changing the reference epoch**.
(For changing epochs, see the *time dependent* tutorial.)

For more on transformations, refer to the [GDA2020 Technical Manual](https://www.anzlic.gov.au/sites/default/files/files/GDA2020%20Technical%20Manual%20V1.8_published.pdf).

## Common Example
The most common datum transformation is converting from **MGA94** to **MGA2020**.
This is handled by a function in the transformation module that:
1) Converts the grid input to Cartesian **(XYZ)**,
2) Runs a **7‑parameter Helmert** transformation using the **GDA94→GDA2020** transformation constant.
We'll break down how this function works later.

**Import GeodePy**

In [ ]:
import geodepy.transform

**Define MGA94 coordinates**

In [ ]:
zone_94 = 55
east_94 = 696053.3373
north_94 = 6086610.1338

**Transform to MGA2020**

In [ ]:
(zone_20, east_20, north_20, _, _) = geodepy.transform.transform_mga94_to_mga2020(
    zone_94, east_94, north_94
)

print(zone_20, east_20, north_20)

This is the MGA2020 coordinates.

## Constructing a new Transformation Function
Now we'll complete a transformation **without a dedicated function** and then wrap it into our own convenient function.
Example: transform from **AGD84** to **GDA2020**.

**Import modules**

In [ ]:
import geodepy.transform
import geodepy.constants
import geodepy.angles

**Starting coordinates (AGD84)**

In [ ]:
lat = geodepy.angles.DMSAngle(-23, 33, 25.21)
long = geodepy.angles.DMSAngle(133, 49, 13.87)
height = 427.863
print(f"The AGD84 position is {lat}, {long}, {height}")

All transformations in GeodePy use **Cartesian (XYZ)** coordinates. Convert LLH to XYZ:

In [ ]:
x, y, z = geodepy.transform.llh2xyz(lat, long, height)
print(x, y, z)

**Transformation parameters** are provided in `geodepy.constants`. If the required transform is missing, you can add one using the *transformation class*. Here we'll use **AGD84→GDA94**.

In [ ]:
print(geodepy.constants.agd84_to_gda94)

Perform a **7‑parameter Helmert** transformation (AGD84→GDA94):

In [ ]:
x_94, y_94, z_94, _ = geodepy.transform.conform7(x, y, z, geodepy.constants.agd84_to_gda94)
print(x_94, y_94, z_94)

> **Tip:** The final return from `conform7` includes a VCV matrix. If you don't pass a VCV in, the returned VCV will be `None`. Use `_` to ignore it when you don't need it.

Now transform **GDA94→GDA2020** using the same method:

In [ ]:
x_20, y_20, z_20, _ = geodepy.transform.conform7(x_94, y_94, z_94, geodepy.constants.gda94_to_gda2020)
print(x_20, y_20, z_20)

Convert Cartesian back to geographic (lat, lon, h):

In [ ]:
lat_20, long_20, height_20 = geodepy.transform.xyz2llh(x_20, y_20, z_20)
print(f"The GDA2020 position is {geodepy.angles.dec2dms(lat_20)}, {geodepy.angles.dec2dms(long_20)}, {height_20}")

**Combine into one function**:

We can now combine this into one convenient function.

In [ ]:
def transform_agd84_to_gda2020(lat, long, height):
    x, y, z = geodepy.transform.llh2xyz(lat, long, height)
    x_94, y_94, z_94, _ = geodepy.transform.conform7(x, y, z, geodepy.constants.agd84_to_gda94)
    x_20, y_20, z_20, _ = geodepy.transform.conform7(x_94, y_94, z_94, geodepy.constants.gda94_to_gda2020)
    return geodepy.transform.xyz2llh(x_20, y_20, z_20)

lat_new, long_new, height_new = transform_agd84_to_gda2020(lat, long, height)
print(f"The GDA2020 position is {geodepy.angles.dec2dms(lat_new)}, {geodepy.angles.dec2dms(long_new)}, {height_new}")